In [8]:
import sys
from pathlib import Path

# Add src to path
src_path = Path.cwd().parent / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

from telescope_mcp.drivers.config import DriverConfig, DriverMode, DriverFactory
from telescope_mcp.devices import init_registry, get_registry, shutdown_registry
from telescope_mcp.devices.camera import CaptureOptions

print("✓ Imports successful")

✓ Imports successful


## Step 1: Initialize Digital Twin Driver

In [9]:
# Configure for digital twin mode
config = DriverConfig(mode=DriverMode.DIGITAL_TWIN)
factory = DriverFactory(config)

# Create camera driver
driver = factory.create_camera_driver()
print(f"✓ Created driver: {type(driver).__name__}")

# Initialize registry with driver
init_registry(driver)
registry = get_registry()
print(f"✓ Initialized registry: {registry}")

✓ Created driver: DigitalTwinCameraDriver
✓ Initialized registry: <CameraRegistry(discovered=0, active=0)>


## Step 2: Discover Cameras

In [10]:
# Discover available cameras
cameras = registry.discover()
print(f"✓ Discovered {len(cameras)} camera(s)")

for cam_id, cam_info in cameras.items():
    print(f"  - {cam_info.name} (ID: {cam_id})")
    print(f"    Max resolution: {cam_info.max_width}x{cam_info.max_height}")

✓ Discovered 2 camera(s)
  - Camera 0 (ID: 0)
    Max resolution: 0x0
  - Camera 1 (ID: 1)
    Max resolution: 0x0


## Step 3: Get Camera Instance

In [11]:
# Get camera 0 (finder camera)
camera = registry.get(camera_id=0, auto_connect=True)
print(f"✓ Got camera: {camera}")
print(f"  Connected: {camera.is_connected()}")

# Get camera info
info = camera.get_info()
print(f"\nCamera Info:")
print(f"  Name: {info['name']}")
print(f"  Resolution: {info['max_width']}x{info['max_height']}")
print(f"  Supports: {', '.join(info.get('supported_modes', []))}")

✓ Got camera: <Camera(Camera 0, connected)>


TypeError: 'bool' object is not callable

## Step 4: Capture Frame

In [ ]:
import cv2
import numpy as np
from IPython.display import Image, display

# Capture with default options
options = CaptureOptions(
    exposure_us=100000,  # 100ms
    gain=50
)

result = camera.capture(options)
print(f"✓ Capture successful")
print(f"  Format: {result['format']}")
print(f"  Size: {len(result['data'])} bytes")
print(f"  Dimensions: {result['width']}x{result['height']}")

# Display image
if result['format'] == 'jpeg':
    display(Image(result['data']))
else:
    # Decode RAW8 to display
    img_array = np.frombuffer(result['data'], dtype=np.uint8)
    img_array = img_array.reshape((result['height'], result['width']))
    _, jpeg_data = cv2.imencode('.jpg', img_array)
    display(Image(jpeg_data.tobytes()))

## Step 5: Test Camera Controls

In [ ]:
# Get control value
gain_value = camera._instance.get_control('GAIN', 'value')
print(f"Current gain: {gain_value}")

# Set control value
camera._instance.set_control('GAIN', value=75)
new_gain = camera._instance.get_control('GAIN', 'value')
print(f"New gain: {new_gain}")

# Get control info
gain_info = camera._instance.get_control('GAIN')
print(f"\nGain control info:")
for key, value in gain_info.items():
    print(f"  {key}: {value}")

## Step 6: Test Multiple Camera Access

In [ ]:
# Get same camera again - should return cached instance
camera2 = registry.get(camera_id=0)
print(f"Same instance: {camera is camera2}")
print(f"✓ Singleton pattern working correctly")

# Try to get camera 1 if available
if len(cameras) > 1:
    camera_1 = registry.get(camera_id=1, auto_connect=True)
    print(f"\n✓ Got second camera: {camera_1}")
    print(f"  Connected: {camera_1.is_connected()}")

## Step 7: Cleanup

In [ ]:
# Shutdown registry (disconnects all cameras)
shutdown_registry()
print("✓ Registry shutdown complete")

## Results

If all cells executed successfully, the camera integration is working correctly:
- ✓ Digital twin driver creates simulated cameras
- ✓ Registry discovers and manages camera instances
- ✓ Camera captures frames with configurable options
- ✓ Camera controls work (get/set)
- ✓ Singleton pattern ensures one instance per camera_id
- ✓ Cleanup properly disconnects cameras